In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]  # Add channel dimension
x_test = x_test[..., tf.newaxis]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
# Multiscale feature learning
def multiscale_feature_learning(inputs):
    conv_3x3 = layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(inputs)
    conv_5x5 = layers.Conv2D(64, kernel_size=5, padding="same", activation="relu")(inputs)
    conv_7x7 = layers.Conv2D(64, kernel_size=7, padding="same", activation="relu")(inputs)

    combined = layers.Concatenate()([conv_3x3, conv_5x5, conv_7x7])
    return combined


In [6]:
# Attention mechanism for CNNs
def attention_module(inputs):
    # Flatten spatial dimensions into a single sequence dimension
    height, width, channels = inputs.shape[1], inputs.shape[2], inputs.shape[3]
    flattened_inputs = layers.Reshape((height * width, channels))(inputs)

    # Create query, key, value tensors
    query = layers.Dense(channels, activation="relu")(flattened_inputs)
    key = layers.Dense(channels, activation="relu")(flattened_inputs)
    value = layers.Dense(channels, activation="relu")(flattened_inputs)

    # Apply attention
    attention_scores = layers.Attention()([query, key])
    attention_output = layers.Multiply()([attention_scores, value])

    # Reshape back to the original spatial dimensions
    attention_output = layers.Reshape((height, width, channels))(attention_output)
    return attention_output

In [7]:
# Build the model
inputs = layers.Input(shape=(28, 28, 1))

# Multiscale feature learning
x = multiscale_feature_learning(inputs)

# Attention mechanism
x = attention_module(x)

# Classifier head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = models.Model(inputs, outputs)

In [8]:
# Compile and train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │        640 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │      1,664 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 28, 28,    │      3,200 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 28, 28,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 192)              │            │ conv2d_4[0][0],   │
│                     │                   │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 784, 192)  │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 784, 192)  │     37,056 │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 784, 192)  │     37,056 │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 784, 192)  │          0 │ dense_3[0][0],    │
│ (Attention)         │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 784, 192)  │     37,056 │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 784, 192)  │          0 │ attention_1[0][0… │
│ (Multiply)          │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 28, 28,    │          0 │ multiply_1[0][0]  │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 192)       │          0 │ reshape_3[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     24,704 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 10)        │      1,290 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 142,666 (557.29 KB)

 Trainable params: 142,666 (557.29 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

Epoch 1/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 319s 425ms/step - accuracy: 0.4217 - loss: 1.5958 - val_accuracy: 0.8588 - val_loss: 0.4614
Epoch 2/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 326s 435ms/step - accuracy: 0.8629 - loss: 0.4413 - val_accuracy: 0.8625 - val_loss: 0.4235


In [11]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8387 - loss: 0.4816
Test Accuracy: 0.86
